### FDR多重检验校正说明

#### 1. 为什么需要多重检验校正？

在单变量逻辑回归中，我们对**每个特征**都进行了一次假设检验：
- **零假设 (H₀)**: 该特征与目标变量无关
- **显著性水平**: α = 0.05（意味着5%的假阳性率）

**问题**：当进行多次检验时，假阳性（False Positive）会累积！

**示例**：
- 假设有100个特征，且**都与目标无关**（即H₀全部为真）
- 在α=0.05下，期望有 **100 × 0.05 = 5个** 特征会被错误地判定为显著
- 特征越多，假阳性越多！

---

#### 2. FDR（False Discovery Rate）方法

**FDR** 控制的是：在所有被判定为显著的特征中，假阳性的**比例**

$$
\text{FDR} = \frac{\text{假阳性数量}}{\text{所有显著发现数量}}
$$

**Benjamini-Hochberg (BH) 方法**：
1. 将所有p值从小到大排序：$p_{(1)} \leq p_{(2)} \leq \cdots \leq p_{(m)}$
2. 找到最大的 $k$，使得：
   $$
   p_{(k)} \leq \frac{k}{m} \cdot \alpha
   $$
3. 拒绝前 $k$ 个假设（即这些特征显著）

---

#### 3. FDR vs Bonferroni校正

| 方法 | 控制指标 | 严格程度 | 适用场景 |
|------|---------|---------|---------|
| **Bonferroni** | FWER (Family-Wise Error Rate) | 非常严格 | 探索性研究，容忍假阴性 |
| **FDR (BH)** | False Discovery Rate | 相对宽松 | 高维数据，平衡假阳性和假阴性 |

**Bonferroni校正**: $\alpha_{\text{adjusted}} = \frac{\alpha}{m}$
- 例如：100个特征，α=0.05 → 调整后阈值 = 0.0005（过于严格）
- **优点**：强力控制假阳性
- **缺点**：可能丢失真实的显著特征（假阴性高）

**FDR校正**：
- 允许一定比例的假阳性（例如5%）
- **优点**：更高的统计功效（Statistical Power）
- **缺点**：比Bonferroni宽松，但在医学研究中通常更合理

---

#### 4. 如何使用校正后的结果？

本代码提供了**两套特征集**：

**传统方法（未校正）**：
```python
sig_features_01   # p < 0.1
sig_features_005  # p < 0.05
```
- ⚠️ **风险**：包含较多假阳性

**FDR校正方法（推荐）**：
```python
sig_features_fdr       # q < 0.05 (FDR校正)
sig_features_fdr_01    # q < 0.1 (FDR校正)
```
- ✅ **推荐**：控制假阳性率在5%以内

---

#### 5. 结果解读

查看 `univariate_results` DataFrame 的关键列：

| 列名 | 含义 |
|------|------|
| `P_Value` | 原始p值（未校正） |
| `P_Value_FDR` | FDR校正后的q值 |
| `FDR_Significant` | 是否在FDR=0.05下显著 |

**解读示例**：
- 如果 `P_Value = 0.03`，`P_Value_FDR = 0.08`：
  - 未校正：显著 (p<0.05)
  - 校正后：不显著 (q>0.05)
  - **结论**：可能是假阳性，建议剔除

---

#### 6. 推荐实践

**医学研究建议**：
1. **探索性分析**：使用 `sig_features_fdr_01` (q<0.1)
2. **验证性分析**：使用 `sig_features_fdr` (q<0.05)
3. **结合其他方法**：将FDR校正后的特征与LASSO、Boruta结果结合

**报告规范**：
- 报告时应说明是否进行多重检验校正
- 标准格式：`"p-value < 0.05, FDR-corrected"`
- 提供校正前后的特征数量对比

---

#### 7. 参考文献

- Benjamini, Y., & Hochberg, Y. (1995). *Controlling the false discovery rate: a practical and powerful approach to multiple testing*. Journal of the Royal Statistical Society, 57(1), 289-300.
- Storey, J. D., & Tibshirani, R. (2003). *Statistical significance for genomewide studies*. PNAS, 100(16), 9440-9445.

In [ ]:
# =============================================================================
# 可视化：FDR多重检验校正效果
# =============================================================================
import matplotlib.pyplot as plt
import seaborn as sns

print("=" * 80)
print("FDR校正效果可视化")
print("=" * 80)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# ---------------------- 图1: 原始p值 vs FDR校正后q值对比 ----------------------
ax1 = axes[0, 0]

# 绘制散点图
valid_data = univariate_results.dropna(subset=['P_Value', 'P_Value_FDR'])
ax1.scatter(valid_data['P_Value'], valid_data['P_Value_FDR'], 
            alpha=0.6, s=50, c='steelblue', edgecolors='black', linewidths=0.5)

# 添加对角线（y=x）
max_val = max(valid_data['P_Value'].max(), valid_data['P_Value_FDR'].max())
ax1.plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='y=x (无校正)')

# 添加显著性阈值线
ax1.axhline(y=0.05, color='green', linestyle='--', linewidth=2, label='FDR q=0.05')
ax1.axvline(x=0.05, color='orange', linestyle='--', linewidth=2, label='原始 p=0.05')

ax1.set_xlabel('原始 P-Value', fontsize=12, fontweight='bold')
ax1.set_ylabel('FDR校正后 Q-Value', fontsize=12, fontweight='bold')
ax1.set_title('P-Value vs Q-Value (FDR校正)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# 标注：被FDR过滤的点（原始显著，但校正后不显著）
filtered = valid_data[(valid_data['P_Value'] < 0.05) & (valid_data['P_Value_FDR'] >= 0.05)]
if len(filtered) > 0:
    ax1.scatter(filtered['P_Value'], filtered['P_Value_FDR'], 
                color='red', s=80, marker='x', linewidths=2, 
                label=f'被FDR过滤 ({len(filtered)}个)')
    ax1.legend(fontsize=10)

# ---------------------- 图2: 校正前后显著特征数量对比 ----------------------
ax2 = axes[0, 1]

comparison_data = {
    '原始\np<0.05': (valid_data['P_Value'] < 0.05).sum(),
    '原始\np<0.1': (valid_data['P_Value'] < 0.1).sum(),
    'FDR\nq<0.05': (valid_data['P_Value_FDR'] < 0.05).sum(),
    'FDR\nq<0.1': (valid_data['P_Value_FDR'] < 0.1).sum()
}

colors_comparison = ['#3498db', '#5dade2', '#2ecc71', '#58d68d']
bars = ax2.bar(comparison_data.keys(), comparison_data.values(), color=colors_comparison, edgecolor='black')
ax2.set_ylabel('显著特征数量', fontsize=12, fontweight='bold')
ax2.set_title('校正前后显著特征数量对比', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

# 添加数值标签
for bar, count in zip(bars, comparison_data.values()):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
             str(count), ha='center', va='bottom', fontsize=12, fontweight='bold')

# ---------------------- 图3: P值分布直方图（标注FDR阈值）----------------------
ax3 = axes[1, 0]

# 绘制p值分布
ax3.hist(valid_data['P_Value'], bins=50, alpha=0.7, color='steelblue', 
         edgecolor='black', label='原始 P-Value')

# 添加显著性阈值线
ax3.axvline(x=0.05, color='red', linestyle='--', linewidth=2, label='p=0.05')
ax3.axvline(x=0.1, color='orange', linestyle='--', linewidth=2, label='p=0.1')

# 计算并标注FDR临界p值
# BH方法的临界p值：p_(k) ≤ (k/m) × α
if len(sig_features_fdr) > 0:
    # 找到最后一个FDR显著的特征
    fdr_sig_features = univariate_results[univariate_results['FDR_Significant']]
    if len(fdr_sig_features) > 0:
        fdr_critical_p = fdr_sig_features['P_Value'].max()
        ax3.axvline(x=fdr_critical_p, color='green', linestyle='--', linewidth=2, 
                    label=f'FDR临界p值={fdr_critical_p:.4f}')

ax3.set_xlabel('P-Value', fontsize=12, fontweight='bold')
ax3.set_ylabel('频数', fontsize=12, fontweight='bold')
ax3.set_title('P-Value分布与显著性阈值', fontsize=14, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(axis='y', alpha=0.3)

# ---------------------- 图4: Top 20特征的p值和q值对比 ----------------------
ax4 = axes[1, 1]

# 选择Top 20特征（按原始p值排序）
top20 = univariate_results.head(20).copy()
x_pos = np.arange(len(top20))

# 绘制双条形图
width = 0.35
bars1 = ax4.barh(x_pos - width/2, top20['P_Value'], width, 
                 label='原始 P-Value', color='#3498db', alpha=0.8)
bars2 = ax4.barh(x_pos + width/2, top20['P_Value_FDR'], width, 
                 label='FDR Q-Value', color='#2ecc71', alpha=0.8)

# 设置y轴标签（特征名）
ax4.set_yticks(x_pos)
ax4.set_yticklabels(top20['Feature'], fontsize=9)
ax4.invert_yaxis()  # 最显著的在上面

# 添加显著性阈值线
ax4.axvline(x=0.05, color='red', linestyle='--', linewidth=2, alpha=0.7)

ax4.set_xlabel('P-Value / Q-Value', fontsize=12, fontweight='bold')
ax4.set_title('Top 20特征: P-Value vs Q-Value', fontsize=14, fontweight='bold')
ax4.legend(fontsize=10, loc='lower right')
ax4.grid(axis='x', alpha=0.3)

plt.tight_layout()

# 保存图表
fdr_plot_path = f"{output_dir}/univariate_fdr_correction_comparison.png"
plt.savefig(fdr_plot_path, dpi=300, bbox_inches='tight')
print(f"\n✅ FDR校正可视化图表已保存至: {fdr_plot_path}")
plt.show()

# =============================================================================
# 输出FDR校正的详细统计信息
# =============================================================================
print("\n" + "=" * 80)
print("FDR校正详细统计")
print("=" * 80)

print(f"\n1. 基本信息:")
print(f"   - 总特征数: {len(univariate_results)}")
print(f"   - FDR控制水平: α = {FDR_ALPHA}")

print(f"\n2. 校正前（原始p值）:")
print(f"   - p < 0.05: {len(sig_features_005)} 个")
print(f"   - p < 0.1:  {len(sig_features_01)} 个")

print(f"\n3. 校正后（FDR q值）:")
print(f"   - q < 0.05: {len(sig_features_fdr)} 个 ⭐推荐")
print(f"   - q < 0.1:  {len(sig_features_fdr_01)} 个")

print(f"\n4. 过滤效果:")
filtered_005 = len(sig_features_005) - len(sig_features_fdr)
filtered_01 = len(sig_features_01) - len(sig_features_fdr_01)
print(f"   - 从p<0.05过滤: {filtered_005} 个 ({filtered_005/len(sig_features_005)*100 if len(sig_features_005)>0 else 0:.1f}%)")
print(f"   - 从p<0.1过滤:  {filtered_01} 个 ({filtered_01/len(sig_features_01)*100 if len(sig_features_01)>0 else 0:.1f}%)")

# 找出被FDR过滤掉的特征
if filtered_005 > 0:
    filtered_features = set(sig_features_005) - set(sig_features_fdr)
    print(f"\n5. 被FDR过滤的特征示例（可能的假阳性）:")
    filtered_df = univariate_results[univariate_results['Feature'].isin(filtered_features)].head(5)
    print(filtered_df[['Feature', 'P_Value', 'P_Value_FDR', 'OR']].to_string(index=False))

print("\n" + "=" * 80)
print("✅ FDR校正分析完成")
print("=" * 80)

## 📌 FDR多重检验校正总结

### ✅ 已实现的改进

本notebook已成功集成**FDR（False Discovery Rate）多重检验校正**，用于控制单变量逻辑回归中的假阳性率。

---

### 🔄 工作流程

```
原始数据
    ↓
单变量逻辑回归（每个特征独立建模）
    ↓
计算原始p值（Wald检验）
    ↓
✅ FDR校正（Benjamini-Hochberg方法）
    ↓
生成校正后的q值
    ↓
与LASSO、L1逻辑回归、Boruta结果整合
    ↓
投票机制生成两套特征集：
  - 传统方法（未校正）
  - FDR校正方法（推荐）⭐
```

---

### 📊 新增变量

| 变量名 | 说明 |
|--------|------|
| `P_Value_FDR` | FDR校正后的q值（推荐使用） |
| `FDR_Significant` | 是否在FDR=0.05下显著 |
| `sig_features_fdr` | FDR校正后q<0.05的特征 |
| `core_features_fdr` | FDR校正后被4种方法选中的核心特征 ⭐ |
| `high_confidence_features_fdr` | FDR校正后被≥3种方法选中 ⭐ |

---

### 🎯 推荐使用方法

**医学研究标准实践**：
1. **探索性分析**：使用 `high_confidence_features_fdr`
2. **验证性研究**：使用 `core_features_fdr`
3. **报告格式**：标注为 `"p < 0.05, FDR-corrected"`

**代码示例**：
```python
# 加载特征集
with open('output/feature_selection/selected_feature_sets.json', 'r') as f:
    feature_sets = json.load(f)

# 推荐：使用FDR校正的高置信度特征
selected_features = feature_sets['high_confidence_features_fdr']

# 在后续建模中使用
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]
```

---

### 📈 效果对比

通过可视化分析，您可以查看：
1. **P值 vs Q值散点图**：展示校正效果
2. **显著特征数量对比**：校正前后的差异
3. **Top 20特征对比**：哪些特征被FDR过滤

查看图表：`output/feature_selection/univariate_fdr_correction_comparison.png`

---

### 📚 理论依据

- **方法**：Benjamini-Hochberg (1995) False Discovery Rate控制
- **优势**：比Bonferroni校正更宽松，适合高维数据
- **控制指标**：在所有显著发现中，假阳性比例 ≤ 5%
- **适用场景**：基因组学、生物标志物筛选、特征选择

---

### ⚠️ 注意事项

1. **必须使用FDR校正的特征集**（变量名以`_fdr`结尾）
2. **避免使用未校正的p<0.1特征**（假阳性率高）
3. **报告时说明使用了FDR校正**
4. **校正后可能显著特征减少**（这是正常的，说明过滤掉了假阳性）

---

### 📖 参考文献

Benjamini, Y., & Hochberg, Y. (1995). *Controlling the false discovery rate: a practical and powerful approach to multiple testing*. Journal of the Royal Statistical Society: Series B, 57(1), 289-300.